
<br>

- Após a atualização da camada raw, um notebook Databricks atualizaria a camada refined respeitando definição de schema, criação de mascaramento para CNPJ e ajuste de nomes de campos.

- Obs. O campo de CNPJ, por convenção, manteve-se como string ao invés de numérico como indicado no dicionário.

<br><br>

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW
  tmp_beneficiarios
AS
  SELECT
    cast(ID_CMPT_MOVEL AS CHAR(7)) AS ID_CMPT_MOVEL,
    cast(CD_OPERADORA AS CHAR(6)) AS CD_OPERADORA,
    cast(NM_RAZAO_SOCIAL AS VARCHAR(140)) AS NM_RAZAO_SOCIAL,
    cast(NR_CNPJ AS VARCHAR(14)) AS NR_CNPJ,
    concat(
      substring(cast(NR_CNPJ AS VARCHAR(14)), 1, 2), '.',
      REPEAT('*', 3), '.',
      REPEAT('*', 3), '/',
      substring(cast(NR_CNPJ AS VARCHAR(14)), 9, 4), '-',
      right(cast(NR_CNPJ AS VARCHAR(14)), 2)
    ) AS MSK_NR_CNPJ,
    cast(MODALIDADE_OPERADORA AS VARCHAR(40)) AS MODALIDADE_OPERADORA,
    cast(SG_UF AS CHAR(2)) AS SG_UF,
    cast(CD_MUNICIPIO AS INT) AS CD_MUNICIPIO,
    cast(NM_MUNICIPIO AS VARCHAR(50)) AS NM_MUNICIPIO,
    cast(TP_SEXO AS CHAR(1)) AS TP_SEXO,
    cast(DE_FAIXA_ETARIA AS VARCHAR(16)) AS DE_FAIXA_ETARIA,
    cast(DE_FAIXA_ETARIA_REAJ AS VARCHAR(16)) AS DE_FAIXA_ETARIA_REAJ,
    cast(CD_PLANO AS CHAR(30)) AS CD_PLANO,
    cast(TP_VIGENCIA_PLANO AS CHAR(1)) AS TP_VIGENCIA_PLANO,
    cast(DE_CONTRATACAO_PLANO AS VARCHAR(70)) AS DE_CONTRATACAO_PLANO,
    cast(DE_SEGMENTACAO_PLANO AS VARCHAR(100)) AS DE_SEGMENTACAO_PLANO,
    cast(DE_ABRG_GEOGRAFICA_PLANO AS VARCHAR(60)) AS DE_ABRG_GEOGRAFICA_PLANO,
    cast(COBERTURA_ASSIST_PLAN AS VARCHAR(17)) AS COBERTURA_ASSIST_PLANO,
    cast(TIPO_VINCULO AS VARCHAR(16)) AS TIPO_VINCULO,
    cast(QT_BENEFICIARIO_ATIVO AS INT) AS QT_BENEFICIARIO_ATIVO,
    cast(QT_BENEFICIARIO_ADERIDO AS INT) AS QT_BENEFICIARIO_ADERIDO,
    cast(QT_BENEFICIARIO_CANCELADO AS INT) AS QT_BENEFICIARIO_CANCELADO,
    cast(DT_CARGA AS DATE) AS DT_CARGA
  FROM
    raw.beneficiarios;

In [0]:
%sql

CREATE TABLE
  refined.beneficiarios
USING
  DELTA
AS
  SELECT
    *,
    current_timestamp() AS INGESTION_TS
  FROM
    tmp_beneficiarios;

In [0]:
%sql

DROP VIEW
  tmp_beneficiarios;